<h1> <strong> Calculadora de Distribuciones </strong> </h1>

<div style="background: #f0f0f0; padding: 10px; border-radius: 5px;">
<h2>🚀 Navegación Rápida</h2>
<button onclick="document.getElementById('Discreta Uniforme').scrollIntoView()">Discreta Uniforme</button>
<button onclick="document.getElementById('Binomial').scrollIntoView()">Binomial</button>
</div>

<h2> Importamos las librerias. </h2>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# **Distribuciones Discretas**


## **Distribución uniforme**

In [12]:
# Widgets para parámetros
a_widget = widgets.IntText(description="a:")
b_widget = widgets.IntText(description="b:")

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
        ("P(X ≤ y or X ≥ z)", 7)
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:")
z_widget = widgets.IntText(description="z:")

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_prob(event):
    with salida:
        clear_output()
        a = a_widget.value
        b = b_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if a > b:
            print("Error: a debe ser menor o igual que b.")
            return

        valores = np.arange(a, b + 1)
        n = b - a + 1
        media = (a + b) / 2
        varianza = ((b - a + 1)**2 - 1) / 12

        print(f"Media (μ): {media}")
        print(f"Varianza (σ²): {varianza}")

        def prob_event(condicion):
            count = sum(1 for x in valores if condicion(x))
            return count / n

        if tipo == 1:
            resultado = 1/n if a <= y <= b else 0
        elif tipo == 2:
            resultado = prob_event(lambda x: x > y)
        elif tipo == 3:
            resultado = prob_event(lambda x: x < y)
        elif tipo == 4:
            resultado = prob_event(lambda x: x >= y)
        elif tipo == 5:
            resultado = prob_event(lambda x: x <= y)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser menor o igual que z.")
                return
            resultado = prob_event(lambda x: y <= x <= z)
        elif tipo == 7:
            resultado = prob_event(lambda x: x <= y or x >= z)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado de la probabilidad: {resultado:.4f}")

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob)

# Mostrar los widgets
display(a_widget, b_widget, consulta_widget, y_widget, z_widget, calcular_btn, salida)


IntText(value=0, description='a:')

IntText(value=0, description='b:')

Dropdown(description='Consulta:', options=(('P(X = y)', 1), ('P(X > y)', 2), ('P(X < y)', 3), ('P(X ≥ y)', 4),…

IntText(value=0, description='y:')

IntText(value=0, description='z:')

Button(button_style='success', description='Calcular', style=ButtonStyle())

Output()

## **Binomial**

In [13]:
from scipy.stats import binom

# Widgets para parámetros
n_widget = widgets.IntText(
    description="n (ensayos):",
    min=1,
    value=10
)

p_widget = widgets.FloatText(
    description="p (éxito):",
    min=0.0,
    max=1.0,
    value=0.5
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:", min=0)
z_widget = widgets.IntText(description="z:", min=0)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_prob_binomial(event):
    with salida:
        clear_output()
        n = n_widget.value
        p = p_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if p < 0 or p > 1:
            print("Error: p debe estar entre 0 y 1.")
            return
        if y > n and tipo in [1, 2, 3, 4, 5]:
            print(f"Error: y no puede ser mayor que n ({n}).")
            return

        # Media y varianza
        media = n * p
        varianza = n * p * (1 - p)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculo de probabilidad según el tipo de consulta
        if tipo == 1:
            resultado = binom.pmf(y, n, p)
        elif tipo == 2:
            resultado = 1 - binom.cdf(y, n, p)
        elif tipo == 3:
            resultado = binom.cdf(y - 1, n, p) if y > 0 else 0.0
        elif tipo == 4:
            resultado = 1 - binom.cdf(y - 1, n, p)
        elif tipo == 5:
            resultado = binom.cdf(y, n, p)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = binom.cdf(z, n, p) - binom.cdf(y - 1, n, p) if y > 0 else binom.cdf(z, n, p)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado: {resultado:.6f}")

        # Gráfico de la distribución
        x = np.arange(0, n + 1)
        prob = binom.pmf(x, n, p)

        plt.figure(figsize=(10, 5))
        plt.bar(x, prob, color='skyblue', alpha=0.7)
        plt.title(f'Distribución Binomial (n={n}, p={p})')
        plt.xlabel('Número de éxitos (k)')
        plt.ylabel('Probabilidad')
        plt.grid(True)
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob_binomial)

# Mostrar widgets
display(
    widgets.VBox([
        n_widget,
        p_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **Binomial negativa**

In [11]:
from scipy.stats import nbinom

# Widgets para parámetros
k_widget = widgets.IntText(
    description="k (éxitos deseados):",
    min=1,
    value=5
)

p_widget = widgets.FloatText(
    description="p (prob. éxito):",
    min=0.01,
    max=0.99,
    value=0.5
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:", min=1)
z_widget = widgets.IntText(description="z:", min=1)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_prob_negbinomial(event):
    with salida:
        clear_output()
        k = k_widget.value
        p = p_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if p <= 0 or p >= 1:
            print("Error: p debe estar entre 0 y 1 (exclusivo).")
            return
        if tipo in [1, 2, 3, 4, 5] and y < k:
            print(f"Error: y no puede ser menor que k ({k}).")
            return

        # Media y varianza
        media = k / p
        varianza = k * (1 - p) / (p ** 2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Ajuste para scipy (nbinom usa 'número de fallos antes de k éxitos')
        # X = k + fallos => fallos = x - k
        if tipo == 1:
            resultado = nbinom.pmf(y - k, k, p) if y >= k else 0.0
        elif tipo == 2:
            resultado = 1 - nbinom.cdf(y - k, k, p)
        elif tipo == 3:
            resultado = nbinom.cdf(y - 1 - k, k, p) if y > k else 0.0
        elif tipo == 4:
            resultado = 1 - nbinom.cdf(y - 1 - k, k, p) if y > k else 1.0
        elif tipo == 5:
            resultado = nbinom.cdf(y - k, k, p)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = nbinom.cdf(z - k, k, p) - (nbinom.cdf(y - 1 - k, k, p) if y > k else 0.0)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado: {resultado:.6f}")

        # Gráfico de la distribución (hasta un límite razonable)
        max_x = min(k + 50, int(media + 5 * np.sqrt(varianza))) # Límite dinámico
        x = np.arange(k, max_x + 1)
        prob = nbinom.pmf(x - k, k, p)

        plt.figure(figsize=(10, 5))
        plt.bar(x, prob, color='skyblue', alpha=0.7)
        plt.title(f'Distribución Binomial Negativa (k={k}, p={p})')
        plt.xlabel('Número total de ensayos (X)')
        plt.ylabel('Probabilidad')
        plt.grid(True)
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob_negbinomial)

# Mostrar widgets
display(
    widgets.VBox([
        k_widget,
        p_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **Geométrica**

In [10]:
from scipy.stats import geom

# Widgets para parámetros
p_widget = widgets.FloatText(
    description="p (prob. éxito):",
    min=0.01,
    max=0.99,
    value=0.5
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:", min=1)
z_widget = widgets.IntText(description="z:", min=1)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_prob_geometrica(event):
    with salida:
        clear_output()
        p = p_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if p <= 0 or p >= 1:
            print("Error: p debe estar entre 0 y 1 (exclusivo).")
            return

        # Media y varianza
        media = 1 / p
        varianza = (1 - p) / (p ** 2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculo de probabilidad (scipy.stats.geom usa x >= 1)
        if tipo == 1:
            resultado = geom.pmf(y, p)
        elif tipo == 2:
            resultado = 1 - geom.cdf(y, p)
        elif tipo == 3:
            resultado = geom.cdf(y - 1, p) if y > 1 else 0.0
        elif tipo == 4:
            resultado = 1 - geom.cdf(y - 1, p) if y > 1 else 1.0
        elif tipo == 5:
            resultado = geom.cdf(y, p)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = geom.cdf(z, p) - (geom.cdf(y - 1, p) if y > 1 else 0.0)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado: {resultado:.6f}")

        # Gráfico de la distribución (hasta un límite razonable)
        max_x = min(int(media + 5 * np.sqrt(varianza)), 50)  # Límite dinámico
        x = np.arange(1, max_x + 1)
        prob = geom.pmf(x, p)

        plt.figure(figsize=(10, 5))
        plt.bar(x, prob, color='skyblue', alpha=0.7)
        plt.title(f'Distribución Geométrica (p={p})')
        plt.xlabel('Número de ensayos hasta el primer éxito (X)')
        plt.ylabel('Probabilidad')
        plt.grid(True)
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob_geometrica)

# Mostrar widgets
display(
    widgets.VBox([
        p_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **HiperGeométrica**

In [9]:
from scipy.stats import hypergeom
from math import comb

# Widgets para parámetros
N_widget = widgets.IntText(
    description="N (población):",
    min=1,
    value=50
)

K_widget = widgets.IntText(
    description="K (éxitos en población):",
    min=0,
    value=20
)

n_widget = widgets.IntText(
    description="n (muestra):",
    min=1,
    value=10
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:", min=0)
z_widget = widgets.IntText(description="z:", min=0)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_prob_hipergeometrica(event):
    with salida:
        clear_output()
        N = N_widget.value
        K = K_widget.value
        n = n_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        # Validaciones
        if K > N:
            print("Error: K no puede ser mayor que N.")
            return
        if n > N:
            print("Error: n no puede ser mayor que N.")
            return
        if tipo in [1, 2, 3, 4, 5] and (y > min(n, K) or y < max(0, n + K - N)):
            print(f"Error: y debe estar entre {max(0, n + K - N)} y {min(n, K)}.")
            return

        # Media y varianza
        media = n * (K / N)
        varianza = n * (K / N) * ((N - K) / N) * ((N - n) / (N - 1))

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculo de probabilidad
        if tipo == 1:
            resultado = hypergeom.pmf(y, N, K, n)
        elif tipo == 2:
            resultado = 1 - hypergeom.cdf(y, N, K, n)
        elif tipo == 3:
            resultado = hypergeom.cdf(y - 1, N, K, n) if y > 0 else 0.0
        elif tipo == 4:
            resultado = 1 - hypergeom.cdf(y - 1, N, K, n) if y > 0 else 1.0
        elif tipo == 5:
            resultado = hypergeom.cdf(y, N, K, n)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = hypergeom.cdf(z, N, K, n) - (hypergeom.cdf(y - 1, N, K, n) if y > 0 else 0.0)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado: {resultado:.6f}")

        # Gráfico de la distribución
        x_min = max(0, n + K - N)
        x_max = min(n, K)
        x = np.arange(x_min, x_max + 1)
        prob = hypergeom.pmf(x, N, K, n)

        plt.figure(figsize=(10, 5))
        plt.bar(x, prob, color='skyblue', alpha=0.7)
        plt.title(f'Distribución Hipergeométrica (N={N}, K={K}, n={n})')
        plt.xlabel('Número de éxitos en la muestra (X)')
        plt.ylabel('Probabilidad')
        plt.grid(True)
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob_hipergeometrica)

# Mostrar widgets
display(
    widgets.VBox([
        N_widget,
        K_widget,
        n_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **Poisson**

In [14]:
from scipy.stats import poisson

# Widgets para parámetros
lambda_widget = widgets.FloatText(
    description="λ (tasa promedio):",
    min=0.1,
    value=3.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X = y)", 1),
        ("P(X > y)", 2),
        ("P(X < y)", 3),
        ("P(X ≥ y)", 4),
        ("P(X ≤ y)", 5),
        ("P(y ≤ X ≤ z)", 6),
    ],
    description="Consulta:"
)

y_widget = widgets.IntText(description="y:", min=0, value=2)
z_widget = widgets.IntText(description="z:", min=0, value=5)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_prob_poisson(event):
    with salida:
        clear_output()
        lambda_ = lambda_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if lambda_ <= 0:
            print("Error: λ debe ser positivo.")
            return

        # Media y varianza (iguales en Poisson)
        media = lambda_
        varianza = lambda_

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculo de probabilidad
        if tipo == 1:
            resultado = poisson.pmf(y, lambda_)
        elif tipo == 2:
            resultado = 1 - poisson.cdf(y, lambda_)
        elif tipo == 3:
            resultado = poisson.cdf(y - 1, lambda_) if y > 0 else 0.0
        elif tipo == 4:
            resultado = 1 - poisson.cdf(y - 1, lambda_) if y > 0 else 1.0
        elif tipo == 5:
            resultado = poisson.cdf(y, lambda_)
        elif tipo == 6:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = poisson.cdf(z, lambda_) - (poisson.cdf(y - 1, lambda_) if y > 0 else 0.0)
        else:
            print("Opción no válida.")
            return

        print(f"\nResultado: {resultado:.6f}")

        # Gráfico de la distribución
        x_max = min(int(lambda_ + 5 * np.sqrt(lambda_)), 50)  # Límite dinámico
        x = np.arange(0, x_max + 1)
        prob = poisson.pmf(x, lambda_)

        plt.figure(figsize=(10, 5))
        plt.bar(x, prob, color='skyblue', alpha=0.7)
        plt.title(f'Distribución de Poisson (λ={lambda_})')
        plt.xlabel('Número de eventos (X)')
        plt.ylabel('Probabilidad')
        plt.grid(True)
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_prob_poisson)

# Mostrar widgets
display(
    widgets.VBox([
        lambda_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

# **Distribuciones continuas**

## **Continua Uniforme**

In [15]:
from scipy.stats import uniform


# Widgets para parámetros
a_widget = widgets.FloatText(
    description="a (límite inferior):",
    value=0.0,
    step=0.1
)

b_widget = widgets.FloatText(
    description="b (límite superior):",
    value=1.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(y ≤ X ≤ z)", 1),
        ("P(X ≤ z)", 2),
        ("P(X ≥ y)", 3),
        ("PDF en y", 4),
    ],
    description="Consulta:"
)

y_widget = widgets.FloatText(description="y:", value=0.2)
z_widget = widgets.FloatText(description="z:", value=0.8)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_uniforme_continua(event):
    with salida:
        clear_output()
        a = a_widget.value
        b = b_widget.value
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        if a >= b:
            print("Error: a debe ser menor que b.")
            return

        # Media y varianza
        media = (a + b) / 2
        varianza = (b - a)**2 / 12

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculos
        if tipo == 1:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = uniform.cdf(z, a, b - a) - uniform.cdf(y, a, b - a)
        elif tipo == 2:
            resultado = uniform.cdf(z, a, b - a)
        elif tipo == 3:
            resultado = 1 - uniform.cdf(y, a, b - a)
        elif tipo == 4:
            resultado = uniform.pdf(y, a, b - a) if a <= y <= b else 0.0
        else:
            print("Opción no válida.")
            return

        if tipo != 4:
            print(f"\nResultado: {resultado:.6f}")
        else:
            print(f"\nPDF en y: {resultado:.6f}")

        # Gráfico
        x = np.linspace(a - (b - a)*0.1, b + (b - a)*0.1, 500)
        pdf = uniform.pdf(x, a, b - a)

        plt.figure(figsize=(10, 5))
        plt.plot(x, pdf, 'b-', linewidth=2, label='PDF')

        # Sombrear áreas según la consulta
        if tipo == 1 and y <= z:
            mask = (x >= y) & (x <= z)
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5)
        elif tipo == 2:
            mask = x <= z
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5)
        elif tipo == 3:
            mask = x >= y
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5)

        plt.title(f'Distribución Uniforme Continua (a={a}, b={b})')
        plt.xlabel('X')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_uniforme_continua)

# Mostrar widgets
display(
    widgets.VBox([
        a_widget,
        b_widget,
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **Normal**

In [3]:
from scipy.stats import norm


# Widgets para parámetros
mu_widget = widgets.FloatText(
    description="μ (media):",
    value=0.0,
    step=0.1
)

sigma_widget = widgets.FloatText(
    description="σ (desviación estándar):",
    min=0.1,
    value=1.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X ≤ x)", 1),
        ("P(X ≥ x)", 2),
        ("P(y ≤ X ≤ x)", 3),
        ("P(X ≤ y o X ≥ x)", 4),
        ("PDF en x", 5),
    ],
    description="Consulta:",
    value=1
)

y_widget = widgets.FloatText(description="y:", value=-1.0, step=0.1)
x_widget = widgets.FloatText(description="x:", value=1.0, step=0.1)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_normal_general(event):
    with salida:
        clear_output()
        mu = mu_widget.value
        sigma = sigma_widget.value
        y = y_widget.value
        x = x_widget.value
        tipo = consulta_widget.value

        if sigma <= 0:
            print("Error: σ debe ser positivo.")
            return

        # Media y varianza
        media = mu
        varianza = sigma**2

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculos
        if tipo == 1:
            resultado = norm.cdf(x, mu, sigma)
            print(f"\nP(X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - norm.cdf(x, mu, sigma)
            print(f"\nP(X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > x:
                print("Error: y debe ser ≤ x.")
                return
            resultado = norm.cdf(x, mu, sigma) - norm.cdf(y, mu, sigma)
            print(f"\nP({y:.2f} ≤ X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = norm.cdf(y, mu, sigma) + (1 - norm.cdf(x, mu, sigma))
            print(f"\nP(X ≤ {y:.2f} o X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = norm.pdf(x, mu, sigma)
            print(f"\nf({x:.2f}) = {resultado:.6f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x_min = mu - 4 * sigma
        x_max = mu + 4 * sigma
        x_vals = np.linspace(x_min, x_max, 500)
        pdf = norm.pdf(x_vals, mu, sigma)

        plt.figure(figsize=(10, 5))
        plt.plot(x_vals, pdf, 'b-', linewidth=2, label=f'PDF (μ={mu}, σ={sigma})')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x_vals <= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≤ {x:.2f})')
        elif tipo == 2:
            mask = x_vals >= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≥ {x:.2f})')
        elif tipo == 3 and y <= x:
            mask = (x_vals >= y) & (x_vals <= x)
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ X ≤ {x:.2f})')
        elif tipo == 4:
            mask1 = x_vals <= y
            mask2 = x_vals >= x
            plt.fill_between(x_vals[mask1], pdf[mask1], color='skyblue', alpha=0.5, label=f'P(X ≤ {y:.2f})')
            plt.fill_between(x_vals[mask2], pdf[mask2], color='orange', alpha=0.5, label=f'P(X ≥ {x:.2f})')

        plt.title('Distribución Normal General')
        plt.xlabel('X')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_normal_general)

# Mostrar widgets
display(
    widgets.VBox([
        mu_widget,
        sigma_widget,
        consulta_widget,
        y_widget,
        x_widget,
        calcular_btn,
        salida
    ])
)

## **Normal Estándar**

In [17]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

# Widgets para parámetros
consulta_widget = widgets.Dropdown(
    options=[
        ("P(Z ≤ z)", 1),
        ("P(Z ≥ z)", 2),
        ("P(y ≤ Z ≤ z)", 3),
        ("P(Z ≤ y o Z ≥ z)", 4),
        ("PDF en z", 5),
    ],
    description="Consulta:",
    value=1
)

y_widget = widgets.FloatText(description="y:", value=-1.0, step=0.1)
z_widget = widgets.FloatText(description="z:", value=1.0, step=0.1)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

# Función para calcular y graficar
def calcular_normal_estandar(event):
    with salida:
        clear_output()
        y = y_widget.value
        z = z_widget.value
        tipo = consulta_widget.value

        # Media y varianza fijas para la estándar
        media = 0.0
        varianza = 1.0

        print(f"Media (μ): {media}")
        print(f"Varianza (σ²): {varianza}")

        # Cálculos
        if tipo == 1:
            resultado = norm.cdf(z)
            print(f"\nP(Z ≤ {z:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - norm.cdf(z)
            print(f"\nP(Z ≥ {z:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > z:
                print("Error: y debe ser ≤ z.")
                return
            resultado = norm.cdf(z) - norm.cdf(y)
            print(f"\nP({y:.2f} ≤ Z ≤ {z:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = norm.cdf(y) + (1 - norm.cdf(z))
            print(f"\nP(Z ≤ {y:.2f} o Z ≥ {z:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = norm.pdf(z)
            print(f"\nϕ({z:.2f}) = {resultado:.6f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x = np.linspace(-4, 4, 500)
        pdf = norm.pdf(x)

        plt.figure(figsize=(10, 5))
        plt.plot(x, pdf, 'b-', linewidth=2, label='PDF (ϕ(z))')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x <= z
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(Z ≤ {z:.2f})')
        elif tipo == 2:
            mask = x >= z
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(Z ≥ {z:.2f})')
        elif tipo == 3 and y <= z:
            mask = (x >= y) & (x <= z)
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ Z ≤ {z:.2f})')
        elif tipo == 4:
            mask1 = x <= y
            mask2 = x >= z
            plt.fill_between(x[mask1], pdf[mask1], color='skyblue', alpha=0.5, label=f'P(Z ≤ {y:.2f})')
            plt.fill_between(x[mask2], pdf[mask2], color='orange', alpha=0.5, label=f'P(Z ≥ {z:.2f})')

        plt.title('Distribución Normal Estándar (μ=0, σ²=1)')
        plt.xlabel('Z')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_normal_estandar)

# Mostrar widgets
display(
    widgets.VBox([
        consulta_widget,
        y_widget,
        z_widget,
        calcular_btn,
        salida
    ])
)

## **Exponencial**

In [4]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.stats import expon
import matplotlib.pyplot as plt

# Widgets para parámetros
lambda_widget = widgets.FloatText(
    description="λ (tasa):",
    min=0.01,
    value=1.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X ≤ x)", 1),
        ("P(X ≥ x)", 2),
        ("P(y ≤ X ≤ x)", 3),
        ("PDF en x", 4),
        ("Percentil p", 5)
    ],
    description="Consulta:",
    value=1
)

x_widget = widgets.FloatText(description="x:", min=0.0, value=1.0, step=0.1)
y_widget = widgets.FloatText(description="y:", min=0.0, value=0.5, step=0.1)
p_widget = widgets.FloatText(description="p (percentil):", min=0.01, max=0.99, value=0.5, step=0.01)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_exponencial(event):
    with salida:
        clear_output()
        lambda_ = lambda_widget.value
        x = x_widget.value
        y = y_widget.value
        p = p_widget.value
        tipo = consulta_widget.value

        if lambda_ <= 0:
            print("Error: λ debe ser positivo.")
            return

        # Media y varianza
        media = 1 / lambda_
        varianza = 1 / (lambda_ ** 2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculos
        if tipo == 1:
            resultado = expon.cdf(x, scale=1/lambda_)
            print(f"\nP(X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - expon.cdf(x, scale=1/lambda_)
            print(f"\nP(X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > x:
                print("Error: y debe ser ≤ x.")
                return
            resultado = expon.cdf(x, scale=1/lambda_) - expon.cdf(y, scale=1/lambda_)
            print(f"\nP({y:.2f} ≤ X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = expon.pdf(x, scale=1/lambda_)
            print(f"\nf({x:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = expon.ppf(p, scale=1/lambda_)
            print(f"\nPercentil {p:.2f}: x = {resultado:.4f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x_max = max(media + 3 * np.sqrt(varianza), x + 1) if tipo != 5 else max(media + 3 * np.sqrt(varianza), resultado + 1)
        x_vals = np.linspace(0, x_max, 500)
        pdf = expon.pdf(x_vals, scale=1/lambda_)

        plt.figure(figsize=(10, 5))
        plt.plot(x_vals, pdf, 'b-', linewidth=2, label=f'PDF (λ={lambda_})')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x_vals <= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≤ {x:.2f})')
        elif tipo == 2:
            mask = x_vals >= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≥ {x:.2f})')
        elif tipo == 3 and y <= x:
            mask = (x_vals >= y) & (x_vals <= x)
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ X ≤ {x:.2f})')
        elif tipo == 5:
            plt.axvline(resultado, color='red', linestyle='--', label=f'Percentil {p:.2f} = {resultado:.2f}')

        plt.title('Distribución Exponencial')
        plt.xlabel('X (tiempo entre eventos)')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_exponencial)

# Organizar widgets
widgets_to_display = [lambda_widget, consulta_widget]

if consulta_widget.value in [1, 2, 4]:
    widgets_to_display.append(x_widget)
elif consulta_widget.value == 3:
    widgets_to_display.extend([y_widget, x_widget])
elif consulta_widget.value == 5:
    widgets_to_display.append(p_widget)

widgets_to_display.extend([calcular_btn, salida])

display(widgets.VBox(widgets_to_display))

# Actualizar widgets visibles según el tipo de consulta
def update_widgets(change):
    with salida:
        if change['new'] in [1, 2, 4]:
            widgets_to_display = [lambda_widget, consulta_widget, x_widget]
        elif change['new'] == 3:
            widgets_to_display = [lambda_widget, consulta_widget, y_widget, x_widget]
        elif change['new'] == 5:
            widgets_to_display = [lambda_widget, consulta_widget, p_widget]

        clear_output()
        display(widgets.VBox(widgets_to_display + [calcular_btn, salida]))

consulta_widget.observe(update_widgets, names='value')

## **Erlang**

In [5]:
from scipy.stats import gamma
from math import factorial

# Widgets para parámetros
k_widget = widgets.IntText(
    description="k (fases enteras):",
    min=1,
    value=2,
    step=1
)

lambda_widget = widgets.FloatText(
    description="λ (tasa):",
    min=0.01,
    value=1.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X ≤ x)", 1),
        ("P(X ≥ x)", 2),
        ("P(y ≤ X ≤ x)", 3),
        ("PDF en x", 4),
        ("Percentil p", 5)
    ],
    description="Consulta:",
    value=1
)

x_widget = widgets.FloatText(description="x:", min=0.0, value=2.0, step=0.1)
y_widget = widgets.FloatText(description="y:", min=0.0, value=1.0, step=0.1)
p_widget = widgets.FloatText(description="p (percentil):", min=0.01, max=0.99, value=0.5, step=0.01)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_erlang(event):
    with salida:
        clear_output()
        k = k_widget.value
        lambda_ = lambda_widget.value
        x = x_widget.value
        y = y_widget.value
        p = p_widget.value
        tipo = consulta_widget.value

        if lambda_ <= 0 or k < 1:
            print("Error: λ debe ser positivo y k ≥ 1.")
            return

        # Media y varianza
        media = k / lambda_
        varianza = k / (lambda_ ** 2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Usamos gamma de scipy (Erlang es Gamma con k entero)
        if tipo == 1:
            resultado = gamma.cdf(x, k, scale=1/lambda_)
            print(f"\nP(X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - gamma.cdf(x, k, scale=1/lambda_)
            print(f"\nP(X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > x:
                print("Error: y debe ser ≤ x.")
                return
            resultado = gamma.cdf(x, k, scale=1/lambda_) - gamma.cdf(y, k, scale=1/lambda_)
            print(f"\nP({y:.2f} ≤ X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = gamma.pdf(x, k, scale=1/lambda_)
            print(f"\nf({x:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = gamma.ppf(p, k, scale=1/lambda_)
            print(f"\nPercentil {p:.2f}: x = {resultado:.4f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x_max = max(media + 4 * np.sqrt(varianza), x + 1) if tipo != 5 else max(media + 4 * np.sqrt(varianza), resultado + 1)
        x_vals = np.linspace(0, x_max, 500)
        pdf = gamma.pdf(x_vals, k, scale=1/lambda_)

        plt.figure(figsize=(10, 5))
        plt.plot(x_vals, pdf, 'b-', linewidth=2, label=f'PDF (k={k}, λ={lambda_})')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x_vals <= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≤ {x:.2f})')
        elif tipo == 2:
            mask = x_vals >= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≥ {x:.2f})')
        elif tipo == 3 and y <= x:
            mask = (x_vals >= y) & (x_vals <= x)
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ X ≤ {x:.2f})')
        elif tipo == 5:
            plt.axvline(resultado, color='red', linestyle='--', label=f'Percentil {p:.2f} = {resultado:.2f}')

        plt.title('Distribución Erlang (Gamma con k entero)')
        plt.xlabel('X')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_erlang)

# Organizar widgets
widgets_to_display = [k_widget, lambda_widget, consulta_widget]

if consulta_widget.value in [1, 2, 4]:
    widgets_to_display.append(x_widget)
elif consulta_widget.value == 3:
    widgets_to_display.extend([y_widget, x_widget])
elif consulta_widget.value == 5:
    widgets_to_display.append(p_widget)

widgets_to_display.extend([calcular_btn, salida])

display(widgets.VBox(widgets_to_display))

# Actualizar widgets visibles según el tipo de consulta
def update_widgets(change):
    with salida:
        if change['new'] in [1, 2, 4]:
            widgets_to_display = [k_widget, lambda_widget, consulta_widget, x_widget]
        elif change['new'] == 3:
            widgets_to_display = [k_widget, lambda_widget, consulta_widget, y_widget, x_widget]
        elif change['new'] == 5:
            widgets_to_display = [k_widget, lambda_widget, consulta_widget, p_widget]

        clear_output()
        display(widgets.VBox(widgets_to_display + [calcular_btn, salida]))

consulta_widget.observe(update_widgets, names='value')

## **Gamma**

In [6]:
from scipy.stats import gamma


# Widgets para parámetros
alpha_widget = widgets.FloatText(
    description="α (forma):",
    min=0.1,
    value=2.0,
    step=0.1
)

beta_widget = widgets.FloatText(
    description="β (tasa):",
    min=0.01,
    value=1.0,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X ≤ x)", 1),
        ("P(X ≥ x)", 2),
        ("P(y ≤ X ≤ x)", 3),
        ("PDF en x", 4),
        ("Percentil p", 5)
    ],
    description="Consulta:",
    value=1
)

x_widget = widgets.FloatText(description="x:", min=0.01, value=2.0, step=0.1)
y_widget = widgets.FloatText(description="y:", min=0.01, value=1.0, step=0.1)
p_widget = widgets.FloatText(description="p (percentil):", min=0.01, max=0.99, value=0.5, step=0.01)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_gamma(event):
    with salida:
        clear_output()
        alpha = alpha_widget.value
        beta = beta_widget.value
        x = x_widget.value
        y = y_widget.value
        p = p_widget.value
        tipo = consulta_widget.value

        if alpha <= 0 or beta <= 0:
            print("Error: α y β deben ser positivos.")
            return

        # Media y varianza
        media = alpha / beta
        varianza = alpha / (beta ** 2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculos usando scipy.stats.gamma
        # Nota: scipy usa el parámetro de escala (scale = 1/β)
        if tipo == 1:
            resultado = gamma.cdf(x, alpha, scale=1/beta)
            print(f"\nP(X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - gamma.cdf(x, alpha, scale=1/beta)
            print(f"\nP(X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > x:
                print("Error: y debe ser ≤ x.")
                return
            resultado = gamma.cdf(x, alpha, scale=1/beta) - gamma.cdf(y, alpha, scale=1/beta)
            print(f"\nP({y:.2f} ≤ X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = gamma.pdf(x, alpha, scale=1/beta)
            print(f"\nf({x:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = gamma.ppf(p, alpha, scale=1/beta)
            print(f"\nPercentil {p:.2f}: x = {resultado:.4f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x_max = max(media + 4 * np.sqrt(varianza), x + 1) if tipo != 5 else max(media + 4 * np.sqrt(varianza), resultado + 1)
        x_vals = np.linspace(0, x_max, 500)
        pdf = gamma.pdf(x_vals, alpha, scale=1/beta)

        plt.figure(figsize=(10, 5))
        plt.plot(x_vals, pdf, 'b-', linewidth=2, label=f'PDF (α={alpha}, β={beta})')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x_vals <= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≤ {x:.2f})')
        elif tipo == 2:
            mask = x_vals >= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≥ {x:.2f})')
        elif tipo == 3 and y <= x:
            mask = (x_vals >= y) & (x_vals <= x)
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ X ≤ {x:.2f})')
        elif tipo == 5:
            plt.axvline(resultado, color='red', linestyle='--', label=f'Percentil {p:.2f} = {resultado:.2f}')

        plt.title('Distribución Gamma')
        plt.xlabel('X')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_gamma)

# Organizar widgets
widgets_to_display = [alpha_widget, beta_widget, consulta_widget]

if consulta_widget.value in [1, 2, 4]:
    widgets_to_display.append(x_widget)
elif consulta_widget.value == 3:
    widgets_to_display.extend([y_widget, x_widget])
elif consulta_widget.value == 5:
    widgets_to_display.append(p_widget)

widgets_to_display.extend([calcular_btn, salida])

display(widgets.VBox(widgets_to_display))

# Actualizar widgets visibles según el tipo de consulta
def update_widgets(change):
    with salida:
        if change['new'] in [1, 2, 4]:
            widgets_to_display = [alpha_widget, beta_widget, consulta_widget, x_widget]
        elif change['new'] == 3:
            widgets_to_display = [alpha_widget, beta_widget, consulta_widget, y_widget, x_widget]
        elif change['new'] == 5:
            widgets_to_display = [alpha_widget, beta_widget, consulta_widget, p_widget]

        clear_output()
        display(widgets.VBox(widgets_to_display + [calcular_btn, salida]))

consulta_widget.observe(update_widgets, names='value')

## **Weibull**

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.stats import weibull_min
from scipy.special import gamma
import matplotlib.pyplot as plt

# Widgets para parámetros
lambda_widget = widgets.FloatText(
    description="λ (escala):",
    min=0.1,
    value=1.0,
    step=0.1
)

k_widget = widgets.FloatText(
    description="k (forma):",
    min=0.1,
    value=1.5,
    step=0.1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(X ≤ x)", 1),
        ("P(X ≥ x)", 2),
        ("P(y ≤ X ≤ x)", 3),
        ("PDF en x", 4),
        ("Percentil p", 5)
    ],
    description="Consulta:",
    value=1
)

x_widget = widgets.FloatText(description="x:", min=0.01, value=1.5, step=0.1)
y_widget = widgets.FloatText(description="y:", min=0.01, value=0.5, step=0.1)
p_widget = widgets.FloatText(description="p (percentil):", min=0.01, max=0.99, value=0.5, step=0.01)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_weibull(event):
    with salida:
        clear_output()
        lambda_ = lambda_widget.value
        k = k_widget.value
        x = x_widget.value
        y = y_widget.value
        p = p_widget.value
        tipo = consulta_widget.value

        if lambda_ <= 0 or k <= 0:
            print("Error: λ y k deben ser positivos.")
            return

        # Media y varianza (usando función Gamma)
        media = lambda_ * gamma(1 + 1/k)
        varianza = lambda_**2 * (gamma(1 + 2/k) - (gamma(1 + 1/k))**2)

        print(f"Media (μ): {media:.4f}")
        print(f"Varianza (σ²): {varianza:.4f}")

        # Cálculos usando scipy.stats.weibull_min
        # Nota: scipy usa el parámetro de forma c = k y escala = λ
        if tipo == 1:
            resultado = weibull_min.cdf(x, k, scale=lambda_)
            print(f"\nP(X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - weibull_min.cdf(x, k, scale=lambda_)
            print(f"\nP(X ≥ {x:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > x:
                print("Error: y debe ser ≤ x.")
                return
            resultado = weibull_min.cdf(x, k, scale=lambda_) - weibull_min.cdf(y, k, scale=lambda_)
            print(f"\nP({y:.2f} ≤ X ≤ {x:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = weibull_min.pdf(x, k, scale=lambda_)
            print(f"\nf({x:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = weibull_min.ppf(p, k, scale=lambda_)
            print(f"\nPercentil {p:.2f}: x = {resultado:.4f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x_max = max(media + 4 * np.sqrt(varianza), x + 1) if tipo != 5 else max(media + 4 * np.sqrt(varianza), resultado + 1)
        x_vals = np.linspace(0, x_max, 500)
        pdf = weibull_min.pdf(x_vals, k, scale=lambda_)

        plt.figure(figsize=(10, 5))
        plt.plot(x_vals, pdf, 'b-', linewidth=2, label=f'PDF (λ={lambda_}, k={k})')

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x_vals <= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≤ {x:.2f})')
        elif tipo == 2:
            mask = x_vals >= x
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(X ≥ {x:.2f})')
        elif tipo == 3 and y <= x:
            mask = (x_vals >= y) & (x_vals <= x)
            plt.fill_between(x_vals[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ X ≤ {x:.2f})')
        elif tipo == 5:
            plt.axvline(resultado, color='red', linestyle='--', label=f'Percentil {p:.2f} = {resultado:.2f}')

        plt.title('Distribución Weibull')
        plt.xlabel('X')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_weibull)

# Organizar widgets
widgets_to_display = [lambda_widget, k_widget, consulta_widget]

if consulta_widget.value in [1, 2, 4]:
    widgets_to_display.append(x_widget)
elif consulta_widget.value == 3:
    widgets_to_display.extend([y_widget, x_widget])
elif consulta_widget.value == 5:
    widgets_to_display.append(p_widget)

widgets_to_display.extend([calcular_btn, salida])

display(widgets.VBox(widgets_to_display))

# Actualizar widgets visibles según el tipo de consulta
def update_widgets(change):
    with salida:
        if change['new'] in [1, 2, 4]:
            widgets_to_display = [lambda_widget, k_widget, consulta_widget, x_widget]
        elif change['new'] == 3:
            widgets_to_display = [lambda_widget, k_widget, consulta_widget, y_widget, x_widget]
        elif change['new'] == 5:
            widgets_to_display = [lambda_widget, k_widget, consulta_widget, p_widget]

        clear_output()
        display(widgets.VBox(widgets_to_display + [calcular_btn, salida]))

consulta_widget.observe(update_widgets, names='value')

## **T de Student**

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
from scipy.stats import t
import matplotlib.pyplot as plt

# Widgets para parámetros
df_widget = widgets.IntText(
    description="ν (grados libertad):",
    min=1,
    value=10,
    step=1
)

consulta_widget = widgets.Dropdown(
    options=[
        ("P(T ≤ t)", 1),
        ("P(T ≥ t)", 2),
        ("P(y ≤ T ≤ t)", 3),
        ("P(T ≤ y o T ≥ t)", 4),
        ("PDF en t", 5),
        ("Percentil p", 6)
    ],
    description="Consulta:",
    value=1
)

t_widget = widgets.FloatText(description="t:", value=1.5, step=0.1)
y_widget = widgets.FloatText(description="y:", value=-1.0, step=0.1)
p_widget = widgets.FloatText(description="p (percentil):", min=0.01, max=0.99, value=0.95, step=0.01)

calcular_btn = widgets.Button(description="Calcular", button_style='success')
salida = widgets.Output()

def calcular_tstudent(event):
    with salida:
        clear_output()
        df = df_widget.value
        t_val = t_widget.value
        y = y_widget.value
        p = p_widget.value
        tipo = consulta_widget.value

        if df <= 0:
            print("Error: ν debe ser positivo.")
            return

        # Media y varianza
        media = 0 if df > 1 else np.nan
        varianza = df / (df - 2) if df > 2 else np.inf

        print(f"Media (μ): {media:.4f}" if df > 1 else "Media (μ): Indefinida (ν ≤ 1)")
        print(f"Varianza (σ²): {varianza:.4f}" if df > 2 else "Varianza (σ²): Infinita (ν ≤ 2)")

        # Cálculos
        if tipo == 1:
            resultado = t.cdf(t_val, df)
            print(f"\nP(T ≤ {t_val:.2f}) = {resultado:.6f}")
        elif tipo == 2:
            resultado = 1 - t.cdf(t_val, df)
            print(f"\nP(T ≥ {t_val:.2f}) = {resultado:.6f}")
        elif tipo == 3:
            if y > t_val:
                print("Error: y debe ser ≤ t.")
                return
            resultado = t.cdf(t_val, df) - t.cdf(y, df)
            print(f"\nP({y:.2f} ≤ T ≤ {t_val:.2f}) = {resultado:.6f}")
        elif tipo == 4:
            resultado = t.cdf(y, df) + (1 - t.cdf(t_val, df))
            print(f"\nP(T ≤ {y:.2f} o T ≥ {t_val:.2f}) = {resultado:.6f}")
        elif tipo == 5:
            resultado = t.pdf(t_val, df)
            print(f"\nf({t_val:.2f}) = {resultado:.6f}")
        elif tipo == 6:
            resultado = t.ppf(p, df)
            print(f"\nPercentil {p:.2f}: t = {resultado:.4f}")
        else:
            print("Opción no válida.")
            return

        # Gráfico
        x = np.linspace(-4, 4, 500)
        pdf = t.pdf(x, df)

        plt.figure(figsize=(10, 5))
        plt.plot(x, pdf, 'b-', linewidth=2, label=f't(ν={df})')

        # Línea de referencia para la normal estándar (comparación)
        plt.plot(x, norm.pdf(x), 'r--', label='N(0,1)', alpha=0.5)

        # Sombrear áreas según la consulta
        if tipo == 1:
            mask = x <= t_val
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(T ≤ {t_val:.2f})')
        elif tipo == 2:
            mask = x >= t_val
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P(T ≥ {t_val:.2f})')
        elif tipo == 3 and y <= t_val:
            mask = (x >= y) & (x <= t_val)
            plt.fill_between(x[mask], pdf[mask], color='skyblue', alpha=0.5, label=f'P({y:.2f} ≤ T ≤ {t_val:.2f})')
        elif tipo == 4:
            mask1 = x <= y
            mask2 = x >= t_val
            plt.fill_between(x[mask1], pdf[mask1], color='skyblue', alpha=0.5, label=f'P(T ≤ {y:.2f})')
            plt.fill_between(x[mask2], pdf[mask2], color='orange', alpha=0.5, label=f'P(T ≥ {t_val:.2f})')
        elif tipo == 6:
            plt.axvline(resultado, color='red', linestyle='--', label=f'Percentil {p:.2f} = {resultado:.2f}')

        plt.title('Distribución t de Student vs Normal Estándar')
        plt.xlabel('Valor t')
        plt.ylabel('Densidad')
        plt.grid(True)
        plt.legend()
        plt.show()

# Enlazar el botón al evento
calcular_btn.on_click(calcular_tstudent)

# Organizar widgets
widgets_to_display = [df_widget, consulta_widget]

if consulta_widget.value in [1, 2, 5]:
    widgets_to_display.append(t_widget)
elif consulta_widget.value in [3, 4]:
    widgets_to_display.extend([y_widget, t_widget])
elif consulta_widget.value == 6:
    widgets_to_display.append(p_widget)

widgets_to_display.extend([calcular_btn, salida])

display(widgets.VBox(widgets_to_display))

# Actualizar widgets visibles según el tipo de consulta
def update_widgets(change):
    with salida:
        if change['new'] in [1, 2, 5]:
            widgets_to_display = [df_widget, consulta_widget, t_widget]
        elif change['new'] in [3, 4]:
            widgets_to_display = [df_widget, consulta_widget, y_widget, t_widget]
        elif change['new'] == 6:
            widgets_to_display = [df_widget, consulta_widget, p_widget]

        clear_output()
        display(widgets.VBox(widgets_to_display + [calcular_btn, salida]))

consulta_widget.observe(update_widgets, names='value')